<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/naftemboriki_teasers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site και να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = []
  for article in articlesL:
    try:
      title = article.find("h3", {"class": "item-title"}).text.strip()
    except:
      title=""
    try:
      article_url = article.find("h3", {"class": "item-title"}).find('a')["href"]
    except:
      article_url=""
    try:
      time = article.find("time", {"class":"item-published"})
      datetime_str = time.text
      date_str = datetime_str.split("-")[0]
      time_str = datetime_str.split("-")[1]
    except:
      date_str = ""
      time_str = ""
    try:
      section = article.find("span", {"class":"category fw-bold text-primary"}).text.strip()
    except:
      section = ""
    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "section": section}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

In [ ]:
def scrape_naftemporiki_teaser_pages(search_term, start_page_nbr=1, end_page_nbr=1):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του naftemporiki.gr
    # if "search" in scrape_url:
    #   search_term = scrape_url.split("/")[4]
    #   if len(scrape_url.split("/")) > 7:
    #     search_params = scrape_url.split("/")[-1]
    #   else:
    #     search_params = ""
    #   url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
    # else:
    #   category = scrape_url.split("/")[3]
    #   url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"
    url = f"https://www.naftemporiki.gr/page/{page_nbr}/?s={search_term}"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    # articlesL = soup.find_all("div", {"class":"nx-article"})
    articlesL = soup.find("div", {"class":"section-body mb-3"}).find_all("div",{"class":"g-col-12"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")


In [ ]:
def scrape_naftemporiki_teaser_pages(search_term, start_page_nbr=1, end_page_nbr=1):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του naftemporiki.gr
    # if "search" in scrape_url:
    #   search_term = scrape_url.split("/")[4]
    #   if len(scrape_url.split("/")) > 7:
    #     search_params = scrape_url.split("/")[-1]
    #   else:
    #     search_params = ""
    #   url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
    # else:
    #   category = scrape_url.split("/")[3]
    #   url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"
    url = f"https://www.naftemporiki.gr/page/{page_nbr}/?s={search_term}"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    # articlesL = soup.find_all("div", {"class":"nx-article"})
    articlesL = soup.find("div", {"class":"section-body mb-3"}).find_all("div",{"class":"g-col-12"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  # Προσθήκη return statement για να επιστρέψει το dataframe
  return df

In [ ]:
df = scrape_naftemporiki_teaser_pages("εργατικο+ατυχημα", 1, 2) # όρος αναζήτησης, o αριθμός της πρώτης σελίδας του pagination που θέλουμε, ο αριθμός της τελευταίας σελίδας του pagination που θέλουμε
df

Scraping page 1: https://www.naftemporiki.gr/page/1/?s=εργατικο+ατυχημα  https://www.naftemporiki.gr/page/1/?s=εργατικο+ατυχημα was scraped successfully
Scraping page 2: https://www.naftemporiki.gr/page/2/?s=εργατικο+ατυχημα  https://www.naftemporiki.gr/page/2/?s=εργατικο+ατυχημα was scraped successfully
Scraping completed. Fetshed 35 articles.


,title,article_url,date,time,section
0,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα στη Βιομη...,https://www.naftemporiki.gr/society/1964633/th...,04/06/2025,12:27,Κοινωνία
1,Θεσσαλονίκη: Σοβαρό εργατικό ατύχημα – Αυτοκίν...,https://www.naftemporiki.gr/society/1952668/th...,06/05/2025,07:47,Κοινωνία
2,Ηράκλειο: Εργατικό ατύχημα σε επιχείρηση με μά...,https://www.naftemporiki.gr/society/1934740/ir...,19/03/2025,10:44,Κοινωνία
3,Κοζάνη: Εργατικό ατύχημα με 22χρονο σοβαρά τρα...,https://www.naftemporiki.gr/society/1806579/ko...,24/10/2024,18:01,Κοινωνία
4,ΠΟΘΑ: Στάση εργασίας και συγκέντρωση διαμαρτυρ...,https://www.naftemporiki.gr/culture/ekdiloseis...,16/10/2024,13:08,Εκδηλώσεις
5,ΓΣΕΕ: Νέο εργατικό ατύχημα στο λιμάνι του Πειραιά,https://www.naftemporiki.gr/society/1746755/gs...,22/08/2024,15:15,Κοινωνία
6,Εργατικό ατύχημα στο Λιμάνι Πειραιά: Έπεσε πάν...,https://www.naftemporiki.gr/maritime/1746639/l...,22/08/2024,13:23,Ναυτιλία
7,Πήλιο: Εργατικό ατύχημα με εργαζόμενη στην καθ...,https://www.naftemporiki.gr/society/1634672/pi...,08/04/2024,11:55,Κοινωνία
8,Η πρώτη συμφωνία αποζημίωσης για εργατικό ατύχ...,https://www.naftemporiki.gr/stories/1576297/i-...,26/01/2024,06:00,Stories
9,Νότια Κορέα: Εργατικό ατύχημα ο θάνατος αεροσυ...,https://www.naftemporiki.gr/kosmos/1533951/not...,07/11/2023,15:01,Κόσμος


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.to_csv("/content/drive/MyDrive/CSVFILES/naftemporiki_teasers.csv", index=False)